In [43]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import matplotlib.pyplot as plt


(torch.Size([60000, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 28, 28]),
 torch.Size([10000]))

In [69]:
DOWNLOAD_MNIST = False  # 不下载
ROOT = '/root/private/torch_datasets/'

BATCH_SIZE = 500
NUM_WORKERS = 8

LR = 0.001  # 学习率

train_data = torchvision.datasets.MNIST(
    root=ROOT,  # 数据集文件
    train=True,  # 训练集数据
    transform=torchvision.transforms.ToTensor(),  # 变成tensor，做归一化处理
    download=DOWNLOAD_MNIST
)

train_loader = data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
    # num_workers=NUM_WORKERS
)

test_data = torchvision.datasets.MNIST(
    root=ROOT,
    train=False,  # 测试集数据
    transform=torchvision.transforms.ToTensor(),
    download=DOWNLOAD_MNIST
)

test_loader = data.DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=True
    # num_workers=NUM_WORKERS
)

train_data.data.shape, train_data.targets.shape, \
test_data.data.shape, test_data.targets.shape


(torch.Size([60000, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 28, 28]),
 torch.Size([10000]))

In [70]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16*14*14
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(2)  # 32*7*7
        )
        self.out = nn.Linear(
            in_features=32 * 7 * 7,
            out_features=10
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # reshape
        output = self.out(x)
        return output


In [86]:
# 由于test_data没有用transform，所以要手动转换成张量，并进行归一化
test_x = torch.unsqueeze(test_data.data, dim=1).type(torch.FloatTensor).cuda() / 255.
test_y = test_data.targets.cuda()
# test_x.shape, test_y.shape

# device = 'cuda' if torch.cuda.is_available() else cpu
# dev_count = torch.cuda.device_count()
# torch.cuda.set_device(3)
cnn = CNN()
cnn.cuda()

cnn_optim = torch.optim.Adam(cnn.parameters(), lr=LR)
cnn_loss_func = nn.CrossEntropyLoss()

EPOCH=5
for epoch in range(EPOCH):
    for step, (d_x, d_y) in enumerate(train_loader):
        d_x = d_x.cuda()
        d_y = d_y.cuda()
        output = cnn(d_x)
        cnn_loss = cnn_loss_func(output, d_y)
        cnn_optim.zero_grad()
        cnn_loss.backward()
        cnn_optim.step()

    acc_count = torch.FloatTensor([0])
    for t_step, (t_x, t_y) in enumerate(test_loader):
        t_x = t_x.cuda()
        t_y = t_y.cuda()
        t_output = cnn(t_x)
        pred_y = torch.max(t_output, 1)[1].cuda().data
        acc_count += torch.sum(pred_y == t_y).cuda().type(torch.FloatTensor)
    acc = acc_count / float(test_data.targets.shape[0])
    acc = acc.numpy()[0]
    content = 'Epoch: {:d} | Loss: {:.2f} | Accuracy: {:.2%}'
    print(content.format(epoch, cnn_loss.cuda().data, acc))


Epoch: 0 | Loss: 0.15 | Accuracy: 95.58%


Epoch: 1 | Loss: 0.08 | Accuracy: 97.28%


Epoch: 2 | Loss: 0.06 | Accuracy: 97.87%


Epoch: 3 | Loss: 0.07 | Accuracy: 98.46%


Epoch: 4 | Loss: 0.03 | Accuracy: 98.62%


In [90]:
torch.save(cnn, './test_torch/CNNModelOfMNIST.pkl')
torch.save(cnn.state_dict(), './test_torch/CNNModelParamOfMNIST.pkl')
